In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [43]:
df = pd.read_csv(r"/Users/pedro/Desktop/Transformers4Rec/202512_data_train_SC_GMV.csv")

/var/folders/29/d_ht1ntn2vz8kd51r6fn1m3c0000gn/T/ipykernel_4554/678770537.py:1: DtypeWarning: Columns (155,165,167,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"/Users/pedro/Desktop/Transformers4Rec/202512_data_train_SC_GMV.csv")


In [44]:
df.head()

,SIT_SITE_ID,CUS_CUST_ID_SEL,DAY_INTERVAL,CUMULATIVE_CLICKS_PER_CARD,CUMULATIVE_VIEWS_PER_CARD,CARD_KEY_CLICK,MELIDATA_SESSION_ID,SELLER_TIMESTAMP,ITE_ITEM_ID,DOM_DOMAIN_ID,...,Health_Seller_Index,mean_site,sd_site,REWARD_STANDARDIZED,0.025,0.975,NO_EXCLUDE,mean_sc_index,sd_sc_index,Health_Seller_Index_Standardized
0,MLB,2.666014e+08,2025-08-18,6,259.0,PREMIUM_LISTING_TYPE,2852f599-dd27-4da1-8f59-a633106fddcc,2025-08-18T16:09:08.917-0400,3.185804e+09,unknown,...,-2.956192,-0.065361,0.260823,0.050598,-13.36851,37.784567,1.0,3.979266,10.053784,-0.689836
1,MLB,1.153915e+09,2025-07-05,1,1.0,PREMIUM_LISTING_TYPE,dba4d0a5-141e-4d78-a25b-8ebcc2a366e3,2025-07-05T15:32:24.980-0400,4.082711e+09,unknown,...,5.172324,-0.065361,0.260823,-1.289741,-13.36851,37.784567,1.0,3.979266,10.053784,0.118668
2,MLB,1.774566e+09,2025-08-22,1,13.0,PREMIUM_LISTING_TYPE,6315343a-d71b-42c6-1cd5-b712b493c8e0,2025-08-22T13:45:14.718-0400,5.230352e+09,unknown,...,-3.764528,-0.065361,0.260823,0.239092,-13.36851,37.784567,1.0,3.979266,10.053784,-0.770237
3,MLB,2.550074e+08,2025-07-10,25,19.0,PREMIUM_LISTING_TYPE,c762578c-d5b1-4a46-89fb-1c88586f9f03,2025-07-10T21:28:29.243-0400,5.465250e+09,unknown,...,17.992995,-0.065361,0.260823,0.223052,-13.36851,37.784567,1.0,3.979266,10.053784,1.393876
4,MLB,2.682879e+08,2025-07-29,2,35.0,PREMIUM_LISTING_TYPE,a01c62b8-6a1c-4b35-a4cb-84db0ae0c70f,2025-07-29T07:33:02.946-0400,4.113235e+09,unknown,...,11.586530,-0.065361,0.260823,-2.225281,-13.36851,37.784567,1.0,3.979266,10.053784,0.756657


In [54]:
df['CARD_KEY_CLICK'].value_counts()

CARD_KEY_CLICK
UNDER_REVIEW_CATALOG_OPT_IN                3092
PB_040                                     3092
POST_SALE_RG                               3092
NEWBIE_WARRANTY_HUNTING                    3092
FREE_SHIPPING_OFFER                        3090
SHORTS_MODIFY_ITEM                         3090
CAMPAIGN_3X                                3089
ACTIVATE_FLEX_SHIPPING                     3089
PREMIUM_LISTING_TYPE                       3089
UNDER_REVIEW                               3089
INCOMPLETE_TECHNICAL_SPECIFICATION         3089
PRICING_HIGH_PRICE                         3088
BUYBOX_STATUS_ELEGIBLE                     3088
CUPONES_CDP                                2981
PRICE_MATCHING                             2739
CATALOG_AUTOOPTIN_ITEMS                    2558
BUY_BOX_LOSING_BY_PRICE                    2202
BAD_SHOPPING_EXPERIENCE                    2069
PUSH_AUTOMATIC_PRICING                     1898
REPUTATION_RECOVERY_COLOR                  1818
ACTIVATE_FLEX_SELLER     

In [46]:
# 1. Definimos tus columnas
id_col = 'CUS_CUST_ID_SEL'
time_col = 'SELLER_TIMESTAMP'
action_col = 'CARD_KEY_CLICK'
reward_col = 'REWARD'

state_features = [
    'SIT_SITE_ID', 'SELLER_SEGMENT', 'REP_CURRENT_LEVEL', # Categóricas
    'ORDERS', 'GMV_USD', 'VISITS_30D', 'CVR_30D'          # Numéricas
]

# 2. Filtrar solo lo que nos sirve (para liberar memoria RAM)
subset_cols = [id_col, time_col, action_col, reward_col] + state_features
df_clean = df[subset_cols].copy()

# 3. Ordenar CRUCIAL: Por usuario y por tiempo
df_clean = df_clean.sort_values(by=[id_col, time_col])

# 4. Manejo básico de nulos (IMPORTANTE: El Transformer odia los NaNs)
# Aquí asumo ceros para numéricos y 'UNKNOWN' para textos, pero valídalo.
num_cols = ['ORDERS', 'GMV_USD', 'VISITS_30D', 'CVR_30D']
cat_cols = ['SIT_SITE_ID', 'SELLER_SEGMENT', 'REP_CURRENT_LEVEL']

df_clean[num_cols] = df_clean[num_cols].fillna(0)
df_clean[cat_cols] = df_clean[cat_cols].fillna('UNKNOWN')

print(df_clean.shape, df.shape)

(65345, 11) (65345, 187)


In [47]:
# Creamos una columna 'state_vector' que contiene una lista con los valores del estado para esa fila
# Nota: Esto mezcla números y strings por ahora. Luego lo procesaremos.
df_clean['state_vector'] = df_clean[state_features].values.tolist()

# Vista previa
df_clean[[id_col, 'state_vector']].head()

,CUS_CUST_ID_SEL,state_vector
11287,4079.0,"[MLB, SELLER DEV, light_green, 48.0, 10086.750..."
64582,62706.0,"[MLA, TO, green_silver, 1612.0, 184019.4174906..."
64583,62706.0,"[MLA, TO, green_silver, 1612.0, 184019.4174906..."
21036,76222.0,"[MLA, SELLER DEV, unknown, 0.0, 0.0, 83.0, 0.0..."
4213,83724.0,"[MLB, SELLER DEV, green, 183.0, 6007.626783507..."


In [48]:
# Agrupamos por usuario y recolectamos las listas
trajectories = df_clean.groupby(id_col).agg({
    time_col: list,        # Lista de timestamps
    action_col: list,      # Lista de acciones tomadas (Secuencia de acciones)
    reward_col: list,      # Lista de recompensas recibidas
    'state_vector': list   # Lista de listas (Secuencia de estados)
}).reset_index()

# Renombramos para claridad (similar al estilo del readme/huggingface)
trajectories.rename(columns={
    time_col: 'timestamps',
    action_col: 'actions',
    reward_col: 'rewards',
    'state_vector': 'observations' # En RL, el estado se suele llamar 'observation'
}, inplace=True)

# Veamos cómo quedó un usuario
trajectories.head()

,CUS_CUST_ID_SEL,timestamps,actions,rewards,observations
0,4079.0,[2025-07-12T11:41:38.422-0400],[INCOMPLETE_TECHNICAL_SPECIFICATION],[-0.2868810078430021],"[[MLB, SELLER DEV, light_green, 48.0, 10086.75..."
1,62706.0,"[2025-07-17T15:39:27.270-0400, 2025-07-17T15:3...","[ADD_FLEX_ZONES_V2, ADD_FLEX_ZONES_V2]","[0.1269863370283323, 0.1269863370283323]","[[MLA, TO, green_silver, 1612.0, 184019.417490..."
2,76222.0,[2025-07-14T19:49:46.748-0400],[NEWBIE_WARRANTY_HUNTING],[0.046835830346895],"[[MLA, SELLER DEV, unknown, 0.0, 0.0, 83.0, 0...."
3,83724.0,[2025-06-22T10:47:24.570-0400],[SHORTS_MODIFY_ITEM],[-1.2819225042764668],"[[MLB, SELLER DEV, green, 183.0, 6007.62678350..."
4,105815.0,"[2025-08-19T12:41:27.840-0400, 2025-08-19T12:4...","[PRICING_HIGH_PRICE, PRICING_HIGH_PRICE]","[-0.2414728651790503, 0.6961654067617704]","[[MLA, SELLER DEV, green_silver, 1137.0, 53218..."


In [49]:
import matplotlib.pyplot as plt
import seaborn as sns

# Asumiendo que tu dataframe se llama 'trajectories' y ya hiciste el groupby
# Calculamos la longitud de la lista de rewards para cada usuario
seq_lens = trajectories['rewards'].apply(len)

# 1. Estadísticas Descriptivas (Duro y parejo)
print("Estadísticas de longitud de secuencias:")
print(seq_lens.describe(percentiles=[0.25, 0.5, 0.75, 0.90, 0.95, 0.99]))

Estadísticas de longitud de secuencias:
count    34993.000000
mean         1.867373
std          2.523164
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
90%          3.000000
95%          5.000000
99%         11.000000
max        104.000000
Name: rewards, dtype: float64


In [50]:
# Filtramos para quedarnos solo con trayectorias de largo 2 o más
df_seq = trajectories[trajectories['rewards'].apply(len) >= 2].copy()

# Veamos las nuevas estadísticas
print(f"Usuarios originales: {trajectories.shape[0]}")
print(f"Usuarios con historia (>=2): {df_seq.shape[0]}")
print(f"Porcentaje retenido: {df_seq.shape[0] / trajectories.shape[0] * 100:.2f}%")

# Nueva distribución de longitudes
print("\nEstadísticas de las secuencias filtradas:")
print(df_seq['rewards'].apply(len).describe(percentiles=[0.25, 0.5, 0.75, 0.90, 0.95, 0.99]))

Usuarios originales: 34993
Usuarios con historia (>=2): 11497
Porcentaje retenido: 32.86%

Estadísticas de las secuencias filtradas:
count    11497.000000
mean         3.639993
std          3.833805
min          2.000000
25%          2.000000
50%          2.000000
75%          4.000000
90%          6.000000
95%          9.000000
99%         19.000000
max        104.000000
Name: rewards, dtype: float64


In [56]:
# 1. Filtramos usuarios válidos (>= 2 interacciones) como acordamos
valid_user_ids = df_seq[id_col].unique()
df_final = df_clean[df_clean[id_col].isin(valid_user_ids)].copy()

# --- 0. CODIFICACIÓN DE ACCIONES ---
# Creamos el encoder para mapear strings de acciones a números
le_action = LabelEncoder()
df_final[action_col] = le_action.fit_transform(df_final[action_col])

# Opcional: Guardate el mapeo por si después querés interpretar resultados
mapping = dict(zip(le_action.classes_, le_action.transform(le_action.classes_)))
print(f"Mapeo de acciones: {mapping}")

# --- A. TRATAMIENTO DE NUMÉRICAS ---
# Separamos las variables en dos grupos según su naturaleza

# Grupo 1: Variables de "Conteo/Monto" (Tienen cola larga, valores muy altos)
# Aplicamos Log(x+1) para que 10, 100, 1000, 10000 se vuelvan distancias iguales
heavy_cols = ['ORDERS', 'GMV_USD', 'VISITS_30D']
for col in heavy_cols:
    # Usamos np.log1p que es log(1+x) para evitar log(0)
    df_final[col] = np.log1p(df_final[col])

# Grupo 2: Variables de "Ratio" (Ya son pequeñas)
ratio_cols = ['CVR_30D'] 
# (No necesitan log, ya están entre 0 y 1 o cerca)

# Ahora aplicamos StandardScaler a TODAS las numéricas para centrarlas en 0
# Las redes aprenden más rápido si los datos están centrados en cero
all_num_cols = heavy_cols + ratio_cols
scaler = StandardScaler()
df_final[all_num_cols] = scaler.fit_transform(df_final[all_num_cols])

print("Numéricas procesadas con Log + StandardScaling.")

# --- B. TRATAMIENTO DE CATEGÓRICAS (Dummies) ---
cat_cols = ['SIT_SITE_ID', 'SELLER_SEGMENT', 'REP_CURRENT_LEVEL']

# Usamos get_dummies. 
# drop_first=False para darle más información explicita al modelo, 
# aunque aumente un poco la dimensión.
df_final = pd.get_dummies(df_final, columns=cat_cols, prefix=cat_cols, dtype=float, drop_first=False)

# Identificamos cuáles son las nuevas columnas de estado (Numéricas + Dummies)
# Excluimos ID, Time, Action, Reward de esta lista
exclude_cols = [id_col, time_col, action_col, reward_col, 'state_vector'] # state_vector viejo si existe
state_features_final = [c for c in df_final.columns if c not in exclude_cols]

print(f"Dimension final del vector de estado: {len(state_features_final)}")
print(f"Variables en el estado: {state_features_final}")

# --- C. RE-AGRUPACIÓN ---
# Actualizamos el vector con las nuevas columnas
df_final['observations'] = df_final[state_features_final].values.tolist()

final_dataset = df_final.groupby(id_col).agg({
    time_col: list,
    action_col: list, # OJO: ¿Ya convertiste esto a números con LabelEncoder antes?
    reward_col: list,
    'observations': list
}).reset_index()

# Renombrar
final_dataset.rename(columns={
    time_col: 'timestamps',
    action_col: 'actions',
    reward_col: 'rewards',
}, inplace=True)

print("Dataset listo con Dummies y Scaling Logarítmico.")

Mapeo de acciones: {'ACQUISITION_GENERAL_ML_ADS': np.int64(0), 'ACTIVATE_FLEX_SELLER': np.int64(1), 'ACTIVATE_FLEX_SHIPPING': np.int64(2), 'ADD_FLEX_ZONES_V2': np.int64(3), 'BAD_SHOPPING_EXPERIENCE': np.int64(4), 'BUYBOX_STATUS_ELEGIBLE': np.int64(5), 'BUY_BOX_LOSING_BY_PRICE': np.int64(6), 'CAMPAIGN_3X': np.int64(7), 'CAPACITY_COLECTA': np.int64(8), 'CAPACITY_FLEX': np.int64(9), 'CAPACITY_XDDO': np.int64(10), 'CATALOG_AUTOOPTIN_ITEMS': np.int64(11), 'COMPATIBILITIES_PENDING': np.int64(12), 'CREDIT_FINANCING': np.int64(13), 'CREDIT_PLUS': np.int64(14), 'CREDIT_PLUS_TASA0': np.int64(15), 'CUPONES_CDP': np.int64(16), 'DEALS_TIER_1': np.int64(17), 'DEALS_TIER_2': np.int64(18), 'FBM_DISENGAGEMENT': np.int64(19), 'FBM_DISPOSAL': np.int64(20), 'FBM_IPI_WARNING': np.int64(21), 'FREE_SHIPPING_OFFER': np.int64(22), 'INCOMPLETE_TECHNICAL_SPECIFICATION': np.int64(23), 'MARKETPLACE_CAMPAIGN': np.int64(24), 'METRICAS_DE_STOCK': np.int64(25), 'MIPAGINA': np.int64(26), 'MIPAGINA_2': np.int64(27), 'NE

In [57]:
final_dataset.head()

,CUS_CUST_ID_SEL,timestamps,actions,rewards,observations
0,62706.0,"[2025-07-17T15:39:27.270-0400, 2025-07-17T15:3...","[3, 3]","[0.1269863370283323, 0.1269863370283323]","[[0.7857503820786138, 1.065347193416983, -0.01..."
1,105815.0,"[2025-08-19T12:41:27.840-0400, 2025-08-19T12:4...","[35, 35]","[-0.2414728651790503, 0.6961654067617704]","[[0.6842381250562125, 0.7787730952401777, 0.53..."
2,113570.0,"[2025-07-24T09:50:16.688-0400, 2025-09-12T11:1...","[35, 29]","[-0.5166133670595068, 0.1169561396927816]","[[1.2703287437992625, 1.1007664452421286, 1.04..."
3,149799.0,"[2025-07-12T09:19:23.615-0400, 2025-07-12T09:1...","[16, 16]","[-0.1979284438234524, -0.1979284438234524]","[[-1.3636302977078034, -1.7349421210652867, -1..."
4,172183.0,"[2025-08-24T20:00:14.683-0400, 2025-08-24T20:2...","[2, 39]","[-0.1586788902843809, -0.6929596291833953]","[[0.9270257209489126, 0.7981246796940512, 0.83..."


In [59]:
# Usar le_action.inverse_transform para obtener las acciones a partir de los encodings
# acciones_reales = le_action.inverse_transform(final_dataset['actions'][0])